输入一组记录，根据60回合周期内的数量统计，进行随机森林分类的推理。

设置文件路径

In [130]:
import os
import platform

DIR = os.path.abspath("")

if platform.system().lower() == 'windows':
    FILE_PATH = DIR + './input/qmdata0819/1113/record.csv'
    print(f'系统是windows，路径为：{FILE_PATH}')
elif platform.system().lower() == 'linux':
    FILE_PATH = DIR + '/input/qmdata0819/1113/record.csv'
    print(f'系统是linux，路径为：{FILE_PATH}')

系统是linux，路径为：/home/yammi/桌面/github/yammi_utils/input/qmdata0819/1113/record.csv


读取文件

In [131]:
read_csv = open(FILE_PATH, 'r', encoding='utf8')

raw_data = []
for line in read_csv:
    line = line.strip()
    time_, item_ = line.split(",")
    raw_data.append(item_)

构建一个训练数据集，训练数据集分为21列，前20列是前20回合的结果，最后一列是最新的结果

In [132]:
# 构建一个header
header = []
for i in range(1,21):
    pre_ = 'col_' + str(i)
    header.append(pre_)

header.append('result')

# 构建数据集
train_data = []
line_data = []
for i in range(20,len(raw_data)):
    # 把当前i的前20个记录依次存入line_data
    for j in range(-20,0):
        line_data.append(raw_data[i+j])
    # 把当前i对应的记录存入line_data
    line_data.append(raw_data[i])
    # 把line_data存入train_data
    if len(line_data) == 21:
        train_data.append(line_data)
    else:
        print('Error: line_data数据长度错误')
        print(len(line_data))
    line_data = []

trainset里的数据项都是文本，需要转为数字，否则无法进行训练。  
对trainset里的每一组数据中的物品数量进行统计，如果数量占优，则该物品对应的数值为1,否则为0。  
对于架子鼓、竖琴、萨克斯风、圆号而言，均称为“高倍乐器”。

In [133]:
STR = ['钢琴', '小提琴', '吉他', '贝斯', '高倍乐器']

High_Rate = ['架子鼓', '竖琴', '萨克斯风', '圆号']

convert_traindata = []
for line in train_data:
    counter = [line.count('钢琴'),line.count('小提琴'),line.count('吉他'),line.count('贝斯'),0]
    for each in line:
        if each in High_Rate:
            counter[4] += 1
    compare_list = []
    for i in range(5):
        compare_list.append([STR[i],counter[i]])
    
    compare_list.sort(key=lambda x:x[1],reverse=1)
    compare_list[0][1] = 1
    compare_list[1][1] = 1
    compare_list[2][1] = 0
    compare_list[3][1] = 0
    compare_list[4][1] = -1

    replace_line = []
    for each in line:
        for item in compare_list:
            if each == item[0]:
                t = item[1]
        replace_line.append(t)

    if len(replace_line) == 21:
        convert_traindata.append(replace_line)
    else:
        print("Error！")

convert_traindata

[[1, -1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1],
 [-1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1, 0],
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1, 0, 0],
 [0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1, 0, 0, -1],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1, 0, 0, -1, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1, 0, 0, -1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1, 0, 0, -1, 0, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1, 0, 0, -1, 0, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 1, 0, 0, -1, 0, 0, 1, 1, -1],
 [0, 1, 1, 0, 1, 1, 0, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1],
 [1, 1, -1, 1, 1, -1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0],
 [1, -1, 1, 1, -1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1],
 [-1, 1, 1, -1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1],
 [1, 1, -1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, -1],
 [1, -1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,

构建train_set

In [134]:
import pandas as pd

train_set = pd.DataFrame(convert_traindata, columns=header)
X = train_set.drop(columns='result',axis=1)
y = train_set['result']

构建随机森林分类器

In [135]:
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.model_selection import train_test_split

seed = 42
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.1, shuffle=False, random_state=seed)

train_y.tail()

1309   -1
1310    1
1311    1
1312    1
1313    1
Name: result, dtype: int64

设置RandomisedSearchCv参数

In [136]:
import numpy as np

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
max_depth = [2, 4]
min_samples_split = [2, 5]
min_samples_leaf = [1, 2]
bootstrap = [True, False]

param_grid = {
    'n_estimators':         n_estimators,
    'max_depth':            max_depth,
    'min_samples_split':     min_samples_split,
    'min_samples_leaf':      min_samples_leaf,
    'bootstrap':            bootstrap,
}

print(param_grid)

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


训练随机森林分类器

In [137]:
rf_Model = RandomForestClassifier()

In [144]:
from sklearn.model_selection import GridSearchCV

rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid, cv=10, verbose=2, n_jobs=4)
rf_Grid.fit(train_x, train_y)

rf_Grid.best_params_

Fitting 10 folds for each of 160 candidates, totalling 1600 fits
[CV] END bootstrap=True, max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END bootstrap=True, max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END bootstrap=True, max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END bootstrap=True, max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END bootstrap=True, max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END bootst

{'bootstrap': True,
 'max_depth': 4,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 25}

In [145]:
print(f'Train Accuracy - {rf_Grid.score(train_x, train_y):.3f}')
print(f'Test Accuracy - {rf_Grid.score(test_x, test_y):.3f}')

Train Accuracy - 0.625
Test Accuracy - 0.633


In [143]:
from sklearn.model_selection import RandomizedSearchCV

rf_RandomGrid = RandomizedSearchCV(estimator = rf_Model, param_distributions = param_grid, cv=10, verbose=2, n_jobs=4)
rf_RandomGrid.fit(train_x, train_y)

rf_RandomGrid.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END bootstrap=False, max_depth=2, min_samples_leaf=2, min_samples_split=5, n_estimators=25; total time=   0.1s
[CV] END bootstrap=False, max_depth=2, min_samples_leaf=2, min_samples_split=5, n_estimators=25; total time=   0.1s
[CV] END bootstrap=False, max_depth=2, min_samples_leaf=2, min_samples_split=5, n_estimators=25; total time=   0.1s
[CV] END bootstrap=False, max_depth=2, min_samples_leaf=2, min_samples_split=5, n_estimators=25; total time=   0.1s
[CV] END bootstrap=False, max_depth=2, min_samples_leaf=2, min_samples_split=5, n_estimators=25; total time=   0.1s
[CV] END bootstrap=False, max_depth=2, min_samples_leaf=2, min_samples_split=5, n_estimators=25; total time=   0.1s[CV] END bootstrap=False, max_depth=2, min_samples_leaf=2, min_samples_split=5, n_estimators=25; total time=   0.1s

[CV] END bootstrap=False, max_depth=2, min_samples_leaf=2, min_samples_split=5, n_estimators=25; total time=   0.1s
[CV] END 

{'n_estimators': 17,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_depth': 4,
 'bootstrap': False}

In [146]:
print(f'Train Accuracy: {rf_RandomGrid.score(train_x, train_y):.3f}')
print(f'Test Accuracy: {rf_RandomGrid.score(test_x, test_y):.3f}')

Train Accuracy: 0.628
Test Accuracy: 0.633


存储模型

In [147]:
joblib.dump(rf_Grid, DIR+"/qqmusic/model/rf_grid.m")
joblib.dump(rf_RandomGrid, DIR+"/qqmusic/model/rf_randomised.m")

['/home/yammi/桌面/github/yammi_utils/qqmusic/model/rf_randomised.m']